In [3]:
import torch
import numpy as np
torch.set_default_dtype(torch.float64)

In [4]:
MAX = 100
n = 100

def random(n, M = MAX):
    return 2 * (np.random.random(n) - 0.5) * M

A = torch.zeros([n, n])
for i in range(0, n):
    A[i] = torch.from_numpy(random(n))
b = torch.tensor(random(n))

def Ax_b(x):
    return (torch.matmul(A, x) - b).norm()**2

m = 100
a = torch.zeros([m, n])
for i in range(0, m):
    a[i] = torch.from_numpy(random(n, 1))
    
def log_reg(x):
    s = torch.zeros(1)
    for i in range(0, m):
        s += torch.log(1 + torch.exp(torch.matmul(a[i], x)))
    return s

In [5]:
class Iterations:
    def __init__(self, x, iters):
        self.x = x
        self.iters = iters
        
def caution(x):
    if(x.requires_grad):
        print('input should not require grad\n')
        return True
    return False

def gradient(f, x):
    if(caution(x)):
        return
    x = torch.tensor(x, requires_grad = True)
    return torch.autograd.grad(outputs = f(x), inputs = x)[0].detach()

def gradient2(f, x):
    if(caution(x)):
        return
    x = torch.tensor(x, requires_grad = True)
    grad = torch.autograd.grad(outputs = f(x), inputs = x, create_graph = True)[0]
    grad2 = torch.zeros([x.shape[0], x.shape[0]])
    for i in range(0, x.shape[0]):
        grad2[i] = torch.autograd.grad(outputs = grad[i], inputs = x, create_graph = True)[0]
    return grad2.detach()

def gradient3(f, x):
    if(caution(x)):
        return
    x = torch.tensor(x, requires_grad = True)
    grad = torch.autograd.grad(outputs = f(x), inputs = x, create_graph = True)[0]
    grad2 = torch.zeros([x.shape[0], x.shape[0]])
    for i in range(0, x.shape[0]):
        grad2[i] = torch.autograd.grad(outputs = grad[i], inputs = x, create_graph = True)[0]
    grad3 = torch.zeros([x.shape[0], x.shape[0], x.shape[0]])
    for i in range(0, x.shape[0]):
        for j in range(0, x.shape[0]):
            grad3[i][j] = torch.autograd.grad(outputs = grad2[i][j], inputs = x, create_graph = True)[0]
    return grad3.detach()

In [6]:
def NAG(f, x_0, gradient, L = 1., eps = 1e-3, max_iter = 1e4):
    iterations = [x_0]
    k = 0
    y_0 = x_0
    gradient_x0 = gradient(x_0)
    
    while(len(iterations) <= max_iter):
        k += 1
        
        x_k = y_0 - 1./L * gradient(y_0)
        y_k = x_k + (k - 1.)/(k + 2.) * (x_k - x_0)
        
        iterations.append(x_k)
        gradient_xk = gradient(x_k)
        
        if(gradient_xk.norm() <= eps):
            break
        if(f(x_k) > f(x_0) + torch.matmul(gradient_x0, x_k - x_0) + L/2. * (x_k - x_0).norm()):
            L *= 2
            k -= 1
            continue
       
        gradient_x0 = gradient_xk
        x_0 = x_k
        y_0 = y_k
        
    return Iterations(x_k, iterations)

In [7]:
def b_r_k(x, y, f, x_k, L_3, grad_rk_x, grad2_f_xk):
    
    def r_k(x):
        
        return 0.5 * torch.matmul(torch.matmul(grad2_f_xk, x - x_k), x - x_k) + L_3 * (x - x_k).norm()**4 / 4.
    
    return r_k(y) - r_k(x) - torch.matmul(grad_rk_x, y - x)

def g_phi_tau(x, f, x_k, grad2_f_xk, tau, L_3):
    
    return gradient(f, x_k) + torch.matmul(grad2_f_xk, x - x_k) + 0.5 * 1./tau**2 * (
    gradient(f, x_k + tau * (x - x_k)) + gradient(f, x_k - tau * (x - x_k)) - 2. * gradient(f, x_k)
    ) + L_3 * (x - x_k).norm()**2 * (x - x_k)

In [8]:
def BDGM(f, x_k, L_3 = 1., tau = 1e-5, eps = 1e-10, max_iter = 1e2, axil_eps = 1e1, axil_iter = 1e2):
    iterations = []
    grad2_f_xk = gradient2(f, x_k)
    y_k = x_k
    delt = eps**1.5 / (gradient(f, x_k).norm()**0.5 + grad2_f_xk.norm()**1.5 / L_3**0.5)
    #tau = 3. * delt / (8 * (2 + 2**0.5) * gradient(f, x_k).norm()) 
    S_k = 2. * ((2 + 2**0.5) / L_3 * gradient(f, x_k).norm())**(1./3)
    
    while(len(iterations) <= max_iter):
        iterations.append(y_k)
        
        g = g_phi_tau(y_k, f, x_k, grad2_f_xk, tau, L_3)
        if(g.norm() <= 1./6 * gradient(f, y_k).norm() - delt):
            break
              
        grad_rk_yk = torch.matmul(grad2_f_xk, y_k - x_k) + L_3 * (y_k - x_k).norm()**2 * (y_k - x_k)
        g_brk = lambda y: torch.matmul(g, y - y_k) + 2 * (1 + 1./2**0.5) * b_r_k(y_k, y, f, x_k, L_3, grad_rk_yk, grad2_f_xk)
        grad_g_brk = lambda y: g + 2 * (1 + 1./2**0.5) * (
            torch.matmul(grad2_f_xk, y - x_k) + L_3 * (y - x_k).norm()**2 * (y - x_k) - grad_rk_yk)
        
        y_k = NAG(f = g_brk, x_0 = y_k, gradient = grad_g_brk, eps = axil_eps, max_iter = axil_iter).x
        
        if((y_k - x_k).norm() > S_k):
            L_3 *= 2
            #continue
            
    return Iterations(y_k, iterations)

In [9]:
def bin_srch(f, x_bound, y_bound, max_iter = 1e1):
    iterations = []
    x_0, x_1 = x_bound
    y_0, y_1 = y_bound
    
    while(len(iterations) <= max_iter):
        
        x = (x_0 + x_1) / 2.
        f_x = f(x)
        
        iterations.append(x)
        
        if(f_x >= y_0):
            if(f_x <= y_1):
                return Iterations(x, iterations)
            else:
                x_0 = x
        else:
            x_1 = x

In [10]:
def HyperFast(f, x_0, L_3 = 1., tau = 1e-5, eps = 1e-7, max_iter = 10, axil_eps = 1e1, axil_iter = 1e2):
    iterations = []
    A_k = 0.
    y_k = x_k = x_k_v = torch.zeros(n)
    
    y_k = BDGM(f, x_k_v, L_3, tau, eps, axil_eps = axil_eps, axil_iter = axil_iter).x
    lam_k = (7./12) / (0.75 * L_3 * (y_k - x_k_v).norm()**2)         
    a_k = 0.5 * (lam_k + (lam_k**2 + 4 * lam_k * A_k)**0.5)
    A_k = A_k + a_k
    x_k = x_k - a_k * gradient(f, y_k)
    iterations.append(y_k)
    
    while(len(iterations) <= max_iter):  
        
        xi_t = lambda t: (1 - t)**2 * A_k * (1.5 * L_3) / (2 * t) * (
        BDGM(f, ((1 - t) * x_k + t * y_k), L_3, tau, eps, axil_eps = axil_eps).x - ((1 - t) * x_k + t * y_k)).norm()**2
        t = bin_srch(xi_t, [0., 1.], [0.5, 0.75]).x
        
        lam_k = (1 - t)**2 * A_k / t
        a_k = 0.5 * (lam_k + (lam_k**2 + 4 * lam_k * A_k)**0.5)
        A_k = A_k + a_k
        y_k = BDGM(f, ((1 - t) * x_k + t * y_k), L_3, tau, eps, axil_eps = axil_eps, axil_iter = axil_iter).x
        x_k = x_k - a_k * gradient(f, y_k)
        
        iterations.append(y_k)
        
    return Iterations(y_k, iterations)

In [11]:
x_0 = torch.tensor(random(n))
x_k = torch.tensor(random(n))
y_k = torch.tensor(random(n))

f = log_reg
L_3 = 1.
tau = 1e-5

In [12]:
iters = HyperFast(f, x_0, L_3, tau)
x_hf = iters.x

AttributeError: 'NoneType' object has no attribute 'x'

In [ ]:
x_nag = NAG(f, x_0, lambda x: gradient(f, x), max_iter = 1e5).x

In [ ]:
(x_nag - x_hf).norm(), abs(f(x_nag - f(x_hf)))